In [3]:
import os
def package(package):
    try:
        import package
    except ImportError:
        print("Trying to Install required module")
        cmd = 'python -m pip install '+package 
        os.system(cmd)
        #import package

In [4]:
#making use of the package function here
package('re')
from bs4 import BeautifulSoup
package('unicodedata')
package('spacy')
package('nltk')
import re 
import unicodedata 
import contractions
import spacy 
import nltk
from nltk.stem import PorterStemmer
import numpy as np
ps = PorterStemmer()
nlp = spacy.load('en_core_web_sm')
ps = nltk.porter.PorterStemmer()
nltk.download('punkt')
nltk.download('stopwords')
os.system('python -m spacy download en')

Trying to Install required module
Trying to Install required module
Trying to Install required module
Trying to Install required module
Trying to Install required module


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ln2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ln2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0

In [6]:
!pip install emot --upgrade
!pip install emoji
import emoji
import csv

Requirement already up-to-date: emot in c:\users\ln2\appdata\local\continuum\anaconda3\lib\site-packages (2.1)


In [7]:
def find_hash(text):
    hash = re.findall(r"#(\w+)", s)
    return hash
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)


def spacy_lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


def simple_stemming(text, stemmer=ps):
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9@#\s]' if not remove_digits else r'[^a-zA-Z@#\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def convert_emojis(text):
    return emoji.demojize(text).replace(":"," ").replace(","," ")

def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.#@]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its Abbreviation in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return (' '.join(user_string))

In [8]:
import tqdm

def text_pre_processor(text, html_strip=True, accented_char_removal=True, contraction_expansion=True,
                       text_lower_case=True, text_stemming=False, text_lemmatization=True, 
                       special_char_removal=True, remove_digits=True, stopword_removal=True, 
                       stopword_list=None,emoji_remove = True, translator=False,
                       convert_emojis=False):
    
    # strip HTML
    if html_strip:
        text = strip_html_tags(text)
    
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    if accented_char_removal:
        text = remove_accented_chars(text)
    
    # expand contractions    
    if contraction_expansion:
        text = expand_contractions(text)
        
    
    # lemmatize text
    if text_lemmatization:
        text = spacy_lemmatize_text(text) 
        
    # remove special characters and\or digits    
    if special_char_removal:
        # insert spaces between special characters to isolate them    
        #special_char_pattern = re.compile(r'([{.(-)!}])')
        #text = special_char_pattern.sub(" \\1 ", text)
        text = remove_special_characters(text, remove_digits=remove_digits)  
         
    # stem text
    if text_stemming and not text_lemmatization:
        text = simple_stemming(text)
        
    # lowercase the text    
    if text_lower_case:
        text = text.lower()
        
        
    # remove stopwords
    if stopword_removal:
        text = remove_stopwords(text, is_lower_case=text_lower_case, 
                                stopwords=stopword_list)
    # remove emoji  
    if emoji_remove:
        text = remove_emoji(text)
        
    #convert emojis
    if convert_emojis:
        text = convert_emojis(text)
        
    if translator:
        text = translator(text)
    
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
    
    
    
    return text